In [1]:
import pandas as pd
from os import listdir, path

In [2]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

# 1 - DATA TO BE READ BY POWER BI

In [2]:
config=pd.read_csv('config.csv',index_col='var')
file = config.loc['raw_data'].path

In [3]:
df = pd.read_csv(file)
df = df[df['Province/State'] != '-']

In [4]:
columns = ['Province/State','Country/Region','Date','Confirmed','Active','Recovered','Deaths']
df = df[columns].groupby(['Province/State','Country/Region','Date']).sum().reset_index()

In [5]:
columns = ['Confirmed','Active','Recovered','Deaths']
new_cases = [item + ' new cases' for item in columns]

columns_mov_avg = columns.copy()
columns_mov_avg.extend(new_cases)

mov_avg = 3
columns_mov_avg = [item + ' {}-day mov avg'.format(mov_avg) for item in columns_mov_avg]

df[new_cases] = df.groupby('Province/State')[columns].diff().fillna(value=0)
df_aux = df.groupby('Province/State').rolling(3).mean().fillna(value=0).reset_index()
df_aux.drop(['Province/State','level_1'],axis=1,inplace=True)

df_aux.columns = columns_mov_avg
df = pd.concat([df,df_aux],axis=1)

In [22]:
df.to_excel('province_report.xlsx')

In [14]:
df['Province/State'].loc[df['Country/Region'] == 'Sweden'].unique()

array(['Blekinge', 'Dalarna', 'Gavleborg', 'Gotland', 'Halland',
       'Jamtland Harjedalen', 'Jonkoping', 'Kalmar', 'Kronoberg',
       'Norrbotten', 'Orebro', 'Ostergotland', 'Skane', 'Sormland',
       'Stockholm', 'Uppsala', 'Varmland', 'Vasterbotten',
       'Vasternorrland', 'Vastmanland', 'Vastra Gotaland'], dtype=object)

In [5]:
def world_data_formatter(raw_data):
    '''
    Creates the world data report from the raw data dataframe.
    
    This function works along the raw_data as returned by the
    raw_data_formatter function. Changes in raw_data_formatter
    affect directly this function.
    
    It creates all columns necessary for analysis with Tableau
    from the John Hopkins Data Science Center and it returns a
    new DataFrame object with calculated columns.
    
    Parameters
    ----------
    raw_data: obj, DataFrame
        the raw data DataFrame as returned by the raw_data_formatter
        function.
    '''
    import pandas as pd
    from numpy import inf, NaN, where
    from datetime import datetime
    
    df_by_country = df.groupby(['Country/Region','Date']).sum().reset_index()

    # Calculating the number of days since the 1st case:
    df_by_country['Days_since_1st_case'] = df_by_country['Date']

    countries = df_by_country['Country/Region'].unique()
    for country in countries:
        idx = where(df_by_country['Country/Region'] == country)
        first_date = pd.to_datetime(df_by_country['Date'].loc[min(idx[0])])
        for index in idx[0]:
            date_diff = (pd.to_datetime(df_by_country.at[index,'Days_since_1st_case']) - 
                         first_date).days
            df_by_country.at[index,'Days_since_1st_case'] = date_diff


    # columns over which the calculations will be performed
    root_columns = ['Active','Confirmed','Deaths','Recovered']

    # creating columns of daily percentage of increase in values:
    for col in root_columns:
        col_daily_inc = col + "_daily_%inc_by_country"
        col_new_cases = col + '_new_cases'
        col_new_cases_inc = col + '_new_cases_inc_rate'
        col_new_cases_inc_speed = col + '_new_cases_inc_rate_speed'

        df_by_country[col_new_cases] = (df_by_country[col] - 
                                        df_by_country[col].shift(periods=1)
                                       ).fillna(value=0)
        df_by_country[col_daily_inc] = df_by_country[col].pct_change().replace([inf, NaN], 0)*100

        # 1st derivative of column datas. It represents the rate of change in new cases:
        df_by_country[col_new_cases_inc] = (df_by_country[col_new_cases] - 
                                            df_by_country[col_new_cases].shift(periods=1)
                                           ).fillna(value=0)        

        # 2nd derivative of column datas. It represents the acceleration of the increase rate
        # of the new cases:
        df_by_country[col_new_cases_inc_speed] = (df_by_country[col_new_cases_inc] -
                                                  df_by_country[col_new_cases_inc].shift(periods=1)
                                                  ).fillna(value=0)
        
    return df_by_country

In order to run specific analysis, such as school level estimations, the PISA data files may need to be merged. (Please note that variable names can slightly differ across PISA cycles. The examples below are from the PISA 2015 database.)

To merge the student data file with the school or/and the teacher data file(s), use the country code 3-character (variable name: CNT in the PISA 2015 data file) and the international school ID (variable name: CNTSCHID in the PISA 2015 data file) for performing the merging process.
To merge the student data file with the parent data file, use the country code 3-character (variable name: CNT in the PISA 2015 data file) and the international student ID (variable name: CNTSTUID in the PISA 2015 data file) for performing the merging process.
To merge the student data file with the cognitive or financial literacy data file(s), use the country code 3-character (variable name: CNT in PISA 2015), the international school ID (variable name: CNTSCHID in the PISA 2015 data file) and the international student ID (variable name: CNTSTUID in the PISA 2015 data file) variables for performing the merging process.

https://www.oecd.org/pisa/data/httpoecdorgpisadatabase-instructions.htm

In [2]:
df=pd.read_sas(r'C:\Users\user\Documents\GitHub\COVID-19\python\notebooks\SCH\cy07_msu_sch_qqq.sas7bdat')

In [3]:
df

,CNTRYID,CNT,CNTSCHID,CYC,NatCen,Region,STRATUM,SUBNATIO,OECD,ADMINMODE,...,STUBEHA,TEACHBEHA,SCMCEG,W_SCHGRNRABWT,W_FSTUWT_SCH_SUM,SENWT,VER_DAT,i,M,BOOKID
0,8.0,b'ALB',800002.0,b'07MS',b'000800',800.0,b'ALB0109',b'0080000',0.0,2.0,...,-1.1797,-2.0409,NaN,7.98891,115.10603,24.81933,b' 09MAY19:11:20:57',29.0,NaN,NaN
1,8.0,b'ALB',800003.0,b'07MS',b'000800',800.0,b'ALB0109',b'0080000',0.0,2.0,...,2.1196,2.5150,NaN,6.50492,26.01970,20.20900,b' 09MAY19:11:20:57',29.0,NaN,NaN
2,8.0,b'ALB',800004.0,b'07MS',b'000800',800.0,b'ALB0211',b'0080000',0.0,2.0,...,-0.6199,-0.4415,NaN,6.64198,99.62974,20.63480,b' 09MAY19:11:20:57',29.0,NaN,NaN
3,8.0,b'ALB',800005.0,b'07MS',b'000800',800.0,b'ALB0107',b'0080000',0.0,2.0,...,-0.2682,-0.0452,NaN,5.93762,66.50133,18.44654,b' 09MAY19:11:20:57',29.0,NaN,NaN
4,8.0,b'ALB',800006.0,b'07MS',b'000800',800.0,b'ALB0105',b'0080000',0.0,2.0,...,-1.3196,-2.0409,NaN,14.61508,277.68645,45.40500,b' 09MAY19:11:20:57',29.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21898,983.0,b'QRT',98300802.0,b'07MS',b'064300',64387.0,b'QRT8787',b'9830000',0.0,2.0,...,-0.8655,-0.9138,NaN,8.35477,180.42418,32.18244,b' 01MAY19:20:24:14',29.0,NaN,NaN
21899,983.0,b'QRT',98300803.0,b'07MS',b'064300',64387.0,b'QRT8787',b'9830000',0.0,2.0,...,3.4411,3.7879,NaN,8.35477,42.30265,32.18244,b' 01MAY19:20:24:14',29.0,NaN,NaN
21900,983.0,b'QRT',98300804.0,b'07MS',b'064300',64387.0,b'QRT8787',b'9830000',0.0,2.0,...,1.4176,1.6385,NaN,8.35477,96.42515,32.18244,b' 01MAY19:20:24:14',29.0,NaN,NaN
21901,983.0,b'QRT',98300805.0,b'07MS',b'064300',64387.0,b'QRT8787',b'9830000',0.0,2.0,...,2.6095,1.5152,NaN,8.35477,31.72699,32.18244,b' 01MAY19:20:24:14',29.0,NaN,NaN
